# SDialog dependencies

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython


if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing Ollama (if you are not planning to use Ollama, you can just comment these lines to speed up the installation)
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/qanastek/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

## Locally

Run following commands and then `Restart` your environment.

In [ ]:
%%script false --no-raise-error
%pip install -e ..
%pip show sdialog

# Tutorial 6: Audio generation and room accoustic

In order to run the next steps in a fast manner, we will start from an existing dialog generated using previous tutorials. If you haven't download yet `dialog_demo.json` locally, please download it from our GitHub repository using the following command:

In [ ]:
# If dialog_demo.json is not present, download it
if os.path.exists("dialog_demo.json"):
    print("dialog_demo.json already exists")
else:
    !wget https://raw.githubusercontent.com/qanastek/sdialog/refs/heads/main/misc/audio/dialog_demo.json

In [ ]:
import os
import json
from tqdm import tqdm

In [ ]:
import sdialog
from sdialog import Dialog

# Load dialog

In [ ]:
original_dialog = Dialog.from_file("dialog_demo.json")
original_dialog.print()

# Audio Generation

### Instanciate voices database

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
dummy_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-kokoro")
dummy_voice_database.get_voice(genre="male", age=20)

### Instanciate TTS model

In [ ]:
!pip install -q kokoro>=0.9.4 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [ ]:
from sdialog.audio.tts_engine import KokoroTTS
tts_engine = KokoroTTS()

## Setup stage: Audio Dialog and Audio Pipeline

In [ ]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

Instanciate the audio pipeline in order to use `Kokoro` (`tts_engine`) as the TTS model and save the audios outputs of all the dialogs into the directory `./audio_outputs`.

The voices are sampled from the `dummy_voice_database` based on the persona attributes `age` and `gender`, as assigned during the original textual dialog.

In [ ]:
!git clone https://github.com/cyrta/dscaper.git

In [ ]:
!pip install -e ./dscaper/
!pip install scaper
!pip install sox
!pip install jams

In [ ]:
import scaper
DATA_PATH = "./dscaper_data" # Path where the sound events, utterances and timelines database will be saved
os.makedirs(DATA_PATH, exist_ok=True)

In [ ]:
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
os.makedirs("./audio_outputs", exist_ok=True)
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_engine,
    dscaper=dsc,
    dir_audio="./audio_outputs",
)
# audio_pipeline = AudioPipeline() # Can also be used with default values

In [ ]:
# Populate the sound events database
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

In [ ]:
from sdialog.audio.jsalt import MedicalRoomGenerator, RoomRole

In [ ]:
room = MedicalRoomGenerator().generate(args={"room_type": RoomRole.EXAMINATION})
display(room.to_image())

Perform the inference of the audio pipeline on the previously converted dialog. In this case we will focus on generating the "unprocessed" audio, which consist of the agregation of all utterances from the dialog. Rather than using the dialog identifier as the name of the directory, we are using here a custom directory name `demo_dialog_room_accoustic` which will be saved at `./audio_outputs/demo_dialog_room_accoustic/`. 

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    environment={
        "room": room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
        "background_effect": "ac_noise_minimal"
    },
    do_step_1=True,
    do_step_2=True,
    do_step_3=True,
    dialog_dir_name="demo_dialog_room_accoustic",
    room_name="my_room_config_1"
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepaths)

In [ ]:
from IPython.display import Audio, display

display(Audio(dialog.audio_step_1_filepath, autoplay=False))
display(Audio(dialog.audio_step_2_filepath, autoplay=False))
print("-"*25)
print("- Room Configurations")
print("-"*25)
for config_name in dialog.audio_step_3_filepaths:
    print(f"> Room Configuration: {config_name}")
    display(Audio(dialog.audio_step_3_filepaths[config_name]["audio_path"], autoplay=False))